This notebook contains the code in data scraping.

In [1]:
# Import the dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import time
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [11]:
# Setting the url
url = "https://dojos.info/NewJersey/Counties.aspx"

# Creating the response
response = requests.get(url)

print(response)

# Using beautiful soup
soup = bs(response.text,"lxml")

<Response [200]>


In [12]:
# Grabbing the list of counties in New Jersey
state = soup.find('ul', class_ = "olist")
state

<ul class="olist">
<li><h4>A</h4>
<ol class="c"><li><a href="AtlanticCounty/" title="Martial Arts Schools in Atlantic County, NJ">Atlantic County</a> (31)</li>
</ol>
</li>
<li><h4>B</h4>
<ol class="c"><li><a href="BergenCounty/" title="Martial Arts Schools in Bergen County, NJ">Bergen County</a> (126)</li>
<li><a href="BurlingtonCounty/" title="Martial Arts Schools in Burlington County, NJ">Burlington County</a> (44)</li>
</ol>
</li>
<li><h4>C</h4>
<ol class="c"><li><a href="CamdenCounty/" title="Martial Arts Schools in Camden County, NJ">Camden County</a> (55)</li>
<li><a href="CapeMayCounty/" title="Martial Arts Schools in Cape May County, NJ">Cape May County</a> (8)</li>
<li><a href="CumberlandCounty/" title="Martial Arts Schools in Cumberland County, NJ">Cumberland County</a> (15)</li>
</ol>
</li>
<li><h4>E</h4>
<ol class="c"><li><a href="EssexCounty/" title="Martial Arts Schools in Essex County, NJ">Essex County</a> (67)</li>
</ol>
</li>
<li><h4>G</h4>
<ol class="c"><li><a href="G

In [13]:
# Pulling the individual counties
county = state.find_all("a")
county

[<a href="AtlanticCounty/" title="Martial Arts Schools in Atlantic County, NJ">Atlantic County</a>,
 <a href="BergenCounty/" title="Martial Arts Schools in Bergen County, NJ">Bergen County</a>,
 <a href="BurlingtonCounty/" title="Martial Arts Schools in Burlington County, NJ">Burlington County</a>,
 <a href="CamdenCounty/" title="Martial Arts Schools in Camden County, NJ">Camden County</a>,
 <a href="CapeMayCounty/" title="Martial Arts Schools in Cape May County, NJ">Cape May County</a>,
 <a href="CumberlandCounty/" title="Martial Arts Schools in Cumberland County, NJ">Cumberland County</a>,
 <a href="EssexCounty/" title="Martial Arts Schools in Essex County, NJ">Essex County</a>,
 <a href="GloucesterCounty/" title="Martial Arts Schools in Gloucester County, NJ">Gloucester County</a>,
 <a href="HudsonCounty/" title="Martial Arts Schools in Hudson County, NJ">Hudson County</a>,
 <a href="HunterdonCounty/" title="Martial Arts Schools in Hunterdon County, NJ">Hunterdon County</a>,
 <a hre

In [16]:
# Code to create a list of counties.

county_tag = []
state = soup.find('ul', class_ = "olist")
county = state.find_all("a")
for i in range(len(county)):
    county_tag.append(county[i]['href'])

In [17]:
# To show the county tags.
county_tag

['AtlanticCounty/',
 'BergenCounty/',
 'BurlingtonCounty/',
 'CamdenCounty/',
 'CapeMayCounty/',
 'CumberlandCounty/',
 'EssexCounty/',
 'GloucesterCounty/',
 'HudsonCounty/',
 'HunterdonCounty/',
 'MercerCounty/',
 'MiddlesexCounty/',
 'MonmouthCounty/',
 'MorrisCounty/',
 'OceanCounty/',
 'PassaicCounty/',
 'SalemCounty/',
 'SomersetCounty/',
 'SussexCounty/',
 'UnionCounty/',
 'WarrenCounty/']

In [18]:
county = []
for j in county_tag:
    county_url = f"https://dojos.info/NewJersey/{j}"
    county.append(county_url)

In [19]:
county

['https://dojos.info/NewJersey/AtlanticCounty/',
 'https://dojos.info/NewJersey/BergenCounty/',
 'https://dojos.info/NewJersey/BurlingtonCounty/',
 'https://dojos.info/NewJersey/CamdenCounty/',
 'https://dojos.info/NewJersey/CapeMayCounty/',
 'https://dojos.info/NewJersey/CumberlandCounty/',
 'https://dojos.info/NewJersey/EssexCounty/',
 'https://dojos.info/NewJersey/GloucesterCounty/',
 'https://dojos.info/NewJersey/HudsonCounty/',
 'https://dojos.info/NewJersey/HunterdonCounty/',
 'https://dojos.info/NewJersey/MercerCounty/',
 'https://dojos.info/NewJersey/MiddlesexCounty/',
 'https://dojos.info/NewJersey/MonmouthCounty/',
 'https://dojos.info/NewJersey/MorrisCounty/',
 'https://dojos.info/NewJersey/OceanCounty/',
 'https://dojos.info/NewJersey/PassaicCounty/',
 'https://dojos.info/NewJersey/SalemCounty/',
 'https://dojos.info/NewJersey/SomersetCounty/',
 'https://dojos.info/NewJersey/SussexCounty/',
 'https://dojos.info/NewJersey/UnionCounty/',
 'https://dojos.info/NewJersey/WarrenC

In [6]:
# Launches browser to search
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Kevin\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [7]:
# Visits the url
url = "https://dojos.info/NewJersey/Counties.aspx"
# url = 'https://dojos.info/Acmfs/'
browser.visit(url)
html = browser.html
soup = bs(html, "html.parser")

In [4]:
# Searches the website for the link to the counties
browser.links.find_by_partial_text("County").click()

In [ ]:
# Selects the dojo list to click.
ulist = browser.find_by_css('.dojolist')

In [ ]:
# Click the the link to move to the dojo information.
ulist.find_by_tag('a').click()

In [8]:
# Finds the address for the dojo.
address = soup.find('address')

# 1. This will gather the text only, replacing <br> with "|"
# 2. Replaces "\r\n\t\t" to have the string look cleaner
# 3. Splits the string to make it easier to grab the different parts
clean_address = address.get_text("|")\
.replace("\n\t\t","")\
.split("|")

In [9]:
clean_address

['    2511 Fire Road, Unit 13 A',
 'Egg Harbor Township',
 ', ',
 'NJ',
 ' ',
 '08234',
 'United States',
 '\n']

In [10]:
# Quits out of the browser.
browser.quit()

The Code below is to start pulling information on the dojo

In [ ]:
url2 = 'https://dojos.info/Acmfs/'
response = requests.get(url2)

# Using beautiful soup
soup = bs(response.text,"html.parser")

In [ ]:
address = soup.find('address')
address

In [ ]:
parts = address.find_all("a")

In [ ]:
for i in range(3):
    print(parts[i].text)

In [ ]:
part = soup.find('address', id='text')

In [ ]:
# Finding the phone number.
phone = soup.find("div", class_ = "phone").get_text()
dojo_phone = phone.replace("\r\n\t\t","").split()
dojo_phone[1]

In [ ]:
address.text

In [ ]:
address

Pulls the Address

In [ ]:
new_address = address.get_text("|")

In [ ]:
new_address.replace(" ","")

In [ ]:
first = new_address.split("|")[0]

In [ ]:
new_address.split("|")

In [ ]:
new_address.replace("\r\n\t\t","")

In [ ]:
second = first.replace("\r\n\t\t","")

In [ ]:
# Splits and joins the string.
third = second.split()
fourth = " ".join(third)
fourth

In [ ]:
dojo_name = soup.find("div", id ="pageTop")
dojo_name.find("h1").get_text()

In [ ]:
url2 = 'https://dojos.info/Acmfs/'
response = requests.get(url2)

# Using beautiful soup
soup = bs(response.text,"html.parser")

address = soup.find('address')
new_address = address.get_text("|")
clean_address = new_address.replace("\r\n\t\t","")
parts = clean_address.split("|")

In [ ]:
parts

In [ ]:
address.get_text("|")\
    .replace("\r\n\t\t","")\
    .split("|")